# Create DataFrame of the Sample  Dataset VS Tag Columns

In [190]:
import pandas as pd
from monkeylearn import MonkeyLearn
import json
import numpy as np


#pull MonkeyLearn trained model
ml = MonkeyLearn('b7c9b60a4a1a33752602958fcec16aeedd72bf49')
model_id = 'cl_A2hjXMXV'

#test dataset, will change to dataset provided
data = ["large social gathering indoors", "sewing by myself", "walking in the park", "sleeping in my bed"]


result = ml.classifiers.classify(model_id, data)
r = result.body

#TODO: these might need to be updated...
allDf = pd.DataFrame({'text':pd.Series([], dtype='str'),
                   'inside':pd.Series([], dtype='float'),
                   'outside':pd.Series([], dtype='float'),
                   '> 6 feet distance':pd.Series([], dtype='float'),
                   '< 6 feet distance':pd.Series([], dtype='float'),
                   'touching':pd.Series([], dtype='float'),
                   'low travel exposure':pd.Series([], dtype='float'),
                   'medium travel exposure':pd.Series([], dtype='float'),
                   'high travel exposure':pd.Series([], dtype='float'),
                   '< 4 people':pd.Series([], dtype='float'),
                   '4-25 people':pd.Series([], dtype='float'),
                   '> 25 people':pd.Series([], dtype='float'),
                   })


for ret in r:
    d = ret['classifications']

    df = pd.DataFrame.from_dict(d)
    confidence = df['confidence'].values.reshape(1, len(df))
    columns = df['tag_name']
    df = pd.DataFrame(confidence, columns=columns)
    df['text'] = ret['text']
    
    allDf = allDf.append(df)
    
#initial dataframe manipulation
allDf = allDf.reset_index().fillna(0)
allDf= allDf[['text', 'inside', 'outside', '> 6 feet distance', '< 6 feet distance',
       'touching', 'low travel exposure', 'medium travel exposure',
       'high travel exposure', '< 4 people', '4-25 people', '> 25 people',
       '>1 hour', 'existence of alcohol and kids', 'medium number of cases',
       'mask or social distancing or sanitation', '5 min < duration < 1 hour',
       'existence of alcohol or kids']]
allDf

,text,inside,outside,> 6 feet distance,< 6 feet distance,touching,low travel exposure,medium travel exposure,high travel exposure,< 4 people,4-25 people,> 25 people,>1 hour,existence of alcohol and kids,medium number of cases,mask or social distancing or sanitation,5 min < duration < 1 hour,existence of alcohol or kids
0,large social gathering indoors,0.845,0.000,0.0,0.842,0.0,0.0,0.828,0.0,0.0,0.0,0.826,0.889,0.839,0.825,0.000,0.000,0.000
1,sewing by myself,0.584,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.538,0.000,0.000,0.543,0.000,0.000
2,walking in the park,0.000,0.804,0.0,0.884,0.0,0.0,0.522,0.0,0.0,0.0,0.000,0.000,0.000,0.500,0.537,0.746,0.000
3,sleeping in my bed,0.594,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.503,0.559


# Defining Functions to Manipulate the DataFrame

In [203]:
def bool_filter(x,conf):
    # if value does not reach confidence level threshold, changes confidence level to 0.
     # x = predicted confidence level, y is the confidence level threshold of interest
   
    if x > conf:
        y = 1
    else:
        y = 0
    return(y)


def double_counts(x,y):
    # returns one value as zero if two similar tags are predicted by the model
    # x = lower-scored value, y = higher-scored value 
    # in the event that the confidence level of x = y =! 0, function defaults to higher-valued variable
    
    if x == 0 and y == 0:
        return
    elif x > y:
        y = 0
        return (x,y)
    else:
        x = 0
        return(x,y)
    
    
    


(2, 0)

# Filters Used to Adjust the Values

In [210]:
#filter that selects the numerical columns to apply bool_filter function
bool_index = ['inside', 'outside', '> 6 feet distance',
       '< 6 feet distance', 'touching', 'low travel exposure',
       'medium travel exposure', 'high travel exposure', '< 4 people',
       '4-25 people', '> 25 people', '>1 hour',
       'existence of alcohol and kids', 'medium number of cases',
       'mask or social distancing or sanitation', '5 min < duration < 1 hour',
       'existence of alcohol or kids']


#WORK ON TOMORROW. try and use the two filters to apply the double_counts function across the dataframe
#selects columns to be multiplied by halfscale
# half_scale = ['outside','< 6 feet distance','medium travel exposure', '4-25 people', 
#               '5 min < duration < 1 hour', 'existence of alcohol or kids']
# full_scale = ['inside', 'touching', 'high travel exposure',
#               '> 25 people', '>1 hour', 'existence of alcohol and kids']



ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# Scaling the Values and Finding Risk Score

In [187]:
allDf[bool_index] = allDf[bool_index].applymap(bool_filter)


allDf[half_scale] = allDf[half_scale].multiply(0.5)
allDf['Risk Factor'] = allDf.sum(axis = 1)
allDf

,text,inside,outside,> 6 feet distance,< 6 feet distance,touching,low travel exposure,medium travel exposure,high travel exposure,< 4 people,4-25 people,> 25 people,>1 hour,existence of alcohol and kids,medium number of cases,mask or social distancing or sanitation,5 min < duration < 1 hour,existence of alcohol or kids,Risk Factor
0,large social gathering indoors,0.5,0,0,1,0,0,0.5,0,0,0.0,1,1,1,1,0,0.0,0.0,6.0
1,sewing by myself,0.5,0,0,0,0,0,0.0,0,0,0.0,0,1,0,0,1,0.0,0.0,2.5
2,walking in the park,0.0,1,0,1,0,0,0.5,0,0,0.0,0,0,0,0,1,0.5,0.0,4.0
3,sleeping in my bed,0.5,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0.5,0.5,1.5
